## Python Find All Parent/Child Nodes in Pandas DataFrame

In [1]:
import pandas as pd
df = pd.DataFrame({
    'parent':[0,0,1,1,2,2,3,3,4,4],
    'child':[1,2,3,4,5, 6,6,7,8,9]
})

In [2]:
df

,parent,child
0,0,1
1,0,2
2,1,3
3,1,4
4,2,5
5,2,6
6,3,6
7,3,7
8,4,8
9,4,9


In [3]:
df['parent_id'] = df.index

## Convert DataFrame to networkx

In [4]:
import networkx as nx


g=nx.DiGraph()
g.add_edges_from(df[['parent', 'child']].to_records(index=False))

## Find All Parent/Child Nodes of Node

In [5]:
from networkx.algorithms.traversal.depth_first_search import dfs_tree

x = dfs_tree(g, 1)
x.edges()

OutEdgeView([(1, 3), (1, 4), (3, 6), (3, 7), (4, 8), (4, 9)])

## List All Parent/Child Nodes in DataFrame

In [21]:
def get_descendants(parent):
    descendants = list(dfs_tree(g, parent).edges())
    return [x[1] for x in descendants]

    
df["descendants"] = df["parent"].apply(get_descendants)    

In [22]:
df

,parent,child,parent_id,descendants
0,0,1,0,"[1, 2, 3, 4, 6, 7, 8, 9, 5]"
1,0,2,1,"[1, 2, 3, 4, 6, 7, 8, 9, 5]"
2,1,3,2,"[3, 4, 6, 7, 8, 9]"
3,1,4,3,"[3, 4, 6, 7, 8, 9]"
4,2,5,4,"[5, 6]"
5,2,6,5,"[5, 6]"
6,3,6,6,"[6, 7]"
7,3,7,7,"[6, 7]"
8,4,8,8,"[8, 9]"
9,4,9,9,"[8, 9]"


In [23]:
def get_children(parent_id):
    list_of_children = []

    def dfs(parent_id):
        child_ids = df[df["parent"]==parent_id]["child"]
        if child_ids.empty:
            return 
        for child_id in child_ids:
            list_of_children.append(child_id)
            dfs(child_id)

    dfs(parent_id)
    return list_of_children

df["descendants"] = df["parent"].apply(get_children)
df

,parent,child,parent_id,descendants
0,0,1,0,"[1, 3, 6, 7, 4, 8, 9, 2, 5, 6]"
1,0,2,1,"[1, 3, 6, 7, 4, 8, 9, 2, 5, 6]"
2,1,3,2,"[3, 6, 7, 4, 8, 9]"
3,1,4,3,"[3, 6, 7, 4, 8, 9]"
4,2,5,4,"[5, 6]"
5,2,6,5,"[5, 6]"
6,3,6,6,"[6, 7]"
7,3,7,7,"[6, 7]"
8,4,8,8,"[8, 9]"
9,4,9,9,"[8, 9]"


## Visualize

In [24]:
import networkx as nx

g=nx.DiGraph()
g.add_edges_from(df[['parent', 'child']].to_records(index=False))
p=nx.drawing.nx_pydot.to_pydot(g)
p.write_png('tree.png')

![](tree.png)